In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

## 데이터 수정

In [3]:
for df in [df_train,df_test]:
    # df['date'] = df['date'].apply(lambda x: x[0:8])
    # 일짜까지 되있는걸 월까지
    df['date'] = df['date'].apply(lambda x: x[0:6])
    df['yr_renovated'] = df['yr_renovated'].apply(lambda x: np.nan if x == 0 else x)
    df['yr_renovated'] = df['yr_renovated'].fillna(df['yr_built'])

In [4]:
for df in [df_train,df_test]:
    df['total_rooms'] = df['bedrooms'] + df['bathrooms']
    df['sqft_ratio'] = df['sqft_living'] / df['sqft_lot']
    df['sqft_total_size'] = df['sqft_above'] + df['sqft_basement']
    df['sqft_ratio15'] = df['sqft_living15'] / df['sqft_lot15'] 
    df['is_renovated'] = df['yr_renovated'] - df['yr_built']
    df['is_renovated'] = df['is_renovated'].apply(lambda x: 0 if x == 0 else 1)
    df['date'] = df['date'].astype('int')

In [5]:
df_train['per_price'] = df_train['price']/df_train['sqft_total_size']
zipcode_price = df_train.groupby(['zipcode'])['per_price'].agg({'mean','var'}).reset_index()
df_train = pd.merge(df_train,zipcode_price,how='left',on='zipcode')
df_test = pd.merge(df_test,zipcode_price,how='left',on='zipcode')

for df in [df_train,df_test]:
    df['mean'] = df['mean'] * df['sqft_total_size']
    df['var'] = df['var'] * df['sqft_total_size']

### 월별 + ZIPCODE를 활용한 더미변수 추가하여 테스트
위도 + 경도로 위치값을 대신하고 싶었지만 어떻게 구역을 나눠야 할지 모르기때문에 ZIPCODE로 대신함

In [6]:
df_train_length = len(df_train)
concat_dataset = pd.concat(objs=[df_train, df_test], axis=0)
dataset_preprocessed = pd.get_dummies(concat_dataset, columns=['date', 'zipcode'])
df_train = dataset_preprocessed[:df_train_length]
df_test = dataset_preprocessed[df_train_length:]

C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


## XGboost

In [7]:
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
import xgboost as xgb
from xgboost import XGBClassifier

In [8]:
train_columns = [c for c in df_train.columns if c not in ['id','price','per_price']]
y_reg = df_train['price']

In [9]:
model= xgb.XGBRegressor(
    learning_rate=0.01,
    n_estimators=10000,
    random_state=2019,
    num_boost_round=5000, 
    early_stopping_rounds=4000,
    verbose_eval=3000, 
    show_stdv=False,
    max_depth=20, 
    nthread=5
)

#prepare fit model with cross-validation
folds = KFold(n_splits=10, shuffle=True, random_state=2019)

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train)):
    X_train,X_validation=df_train.iloc[trn_idx][train_columns],df_train.iloc[val_idx][train_columns]
    y_train,y_validation=y_reg.iloc[trn_idx],y_reg.iloc[val_idx]
    
    dtrain = xgb.DMatrix(X_train)
    dtarget=xgb.DMatrix(y_train)
    dtest = xgb.DMatrix(X_validation)
    dtest_target=xgb.DMatrix(y_validation)
    
    model.fit(X_train,y_train
              ,eval_set=[(X_train,y_train),(X_validation,y_validation)]
              ,eval_metric='rmse'
              ,verbose=1000)

[0]	validation_0-rmse:645488	validation_1-rmse:689314


C:\Users\PC\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[1000]	validation_0-rmse:101797	validation_1-rmse:129228
[2000]	validation_0-rmse:91135	validation_1-rmse:124453
[3000]	validation_0-rmse:85431.1	validation_1-rmse:122122
[4000]	validation_0-rmse:80812	validation_1-rmse:120972
[5000]	validation_0-rmse:77188.8	validation_1-rmse:119927
[6000]	validation_0-rmse:74155.8	validation_1-rmse:118732
[7000]	validation_0-rmse:71602.9	validation_1-rmse:117646
[8000]	validation_0-rmse:69361	validation_1-rmse:116653
[9000]	validation_0-rmse:67342.1	validation_1-rmse:116025
[9999]	validation_0-rmse:65498.3	validation_1-rmse:115151
[0]	validation_0-rmse:645874	validation_1-rmse:686032
[1000]	validation_0-rmse:102493	validation_1-rmse:139569
[2000]	validation_0-rmse:92270.6	validation_1-rmse:131216
[3000]	validation_0-rmse:86263.7	validation_1-rmse:127779
[4000]	validation_0-rmse:81381.3	validation_1-rmse:125916
[5000]	validation_0-rmse:77570	validation_1-rmse:125053
[6000]	validation_0-rmse:74433.6	validation_1-rmse:124380
[7000]	validation_0-rmse:716

In [10]:
predictions=model.predict(df_test[train_columns])

submission = pd.read_csv("sample_submission.csv")
submission['price']=predictions
submission.to_csv('submission.csv',index=False)